In [3]:
with open('testset2.txt', 'r', encoding = 'utf-8') as r:
    text = r.read()

In [6]:
!C:\Users\ASUS\Downloads\udpipe-1.2.0-bin\bin-win64\udpipe.exe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\ASUS\Downloads\russian-syntagrus-ud-2_4-190531.udpipe \
< testset2.txt > text.conllu

Loading UDPipe model: done.


In [5]:
trees = []

with open('text.conllu', 'r', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [107]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [6]:
from nltk.parse import DependencyGraph

In [20]:
#Считаем встречаемость всех глаголов текста

all_verbs = {}

for t in trees:
    
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except AssertionError:
        continue
        
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            if g.nodes[n]['lemma'] in all_verbs:
                
                all_verbs[g.nodes[n]['lemma']] += 1
            else:
                all_verbs[g.nodes[n]['lemma']] = 1

In [21]:
#Оставляем глаголы, которые встречаются не менее 50 раз
freq_verbs = dict((k, v) for k, v in all_verbs.items() if v >= 50)

In [31]:
import operator
sorted(freq_verbs.items(), key=operator.itemgetter(1), reverse = True)

[('обвинить', 461),
 ('мочь', 349),
 ('признать', 343),
 ('подать', 274),
 ('заявить', 258),
 ('сообщить', 216),
 ('стать', 213),
 ('удовлетворить', 173),
 ('требовать', 172),
 ('вынести', 152),
 ('считать', 152),
 ('принять', 151),
 ('просить', 143),
 ('обратиться', 139),
 ('говорить', 139),
 ('быть', 136),
 ('получить', 125),
 ('приговорить', 122),
 ('обвинять', 119),
 ('отказаться', 117),
 ('находиться', 117),
 ('являться', 113),
 ('направить', 105),
 ('обжаловать', 102),
 ('рассматривать', 99),
 ('сказать', 98),
 ('напомнить', 97),
 ('предъявить', 97),
 ('иметь', 94),
 ('подтвердить', 91),
 ('рассмотреть', 90),
 ('пытаться', 89),
 ('решить', 86),
 ('отменить', 83),
 ('арестовывать', 82),
 ('делать', 77),
 ('начаться', 74),
 ('объявить', 72),
 ('утверждать', 71),
 ('дать', 70),
 ('отклонить', 69),
 ('передать', 68),
 ('нет', 68),
 ('оспорить', 68),
 ('назначить', 63),
 ('провести', 63),
 ('согласиться', 62),
 ('взыскать', 62),
 ('оказаться', 62),
 ('удаться', 61),
 ('оставить', 61),

In [110]:
accs = []

for i, tree in enumerate(trees):
    
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        for t in g.triples():
            if 'VERB' in t[0] and 'NOUN' in t[2] and t[1] == 'obj':
                
                if t[0][0] in freq_verbs:
                    
                    verb = m.parse(t[0][0])[0].normal_form
                    noun = m.parse(t[2][0])[0].normal_form
                    accs.append((verb, noun))
                    
    except (AssertionError, TypeError):
        continue

In [112]:
accs

[('признать', 'договор'),
 ('вынести', 'приговор'),
 ('признать', 'ответственность'),
 ('выплатить', 'компенсация'),
 ('обжаловать', 'санкция'),
 ('удовлетворить', 'жалоба'),
 ('взыскать', 'деньга'),
 ('обжаловать', 'санкция'),
 ('оспаривать', 'решение'),
 ('оспаривать', 'решение'),
 ('дать', 'возможность'),
 ('взыскать', 'задолженность'),
 ('обжаловать', 'решение'),
 ('получить', 'средство'),
 ('предъявить', 'обвинение'),
 ('получить', 'комментарий'),
 ('получить', 'комментарий'),
 ('обжаловать', 'решение'),
 ('дать', 'показание'),
 ('признать', 'организация'),
 ('запретить', 'деятельность'),
 ('признать', 'организация'),
 ('запретить', 'деятельность'),
 ('получить', 'компромат'),
 ('обязать', 'ответчик'),
 ('получить', 'информация'),
 ('вынести', 'приговор'),
 ('признать', 'право'),
 ('оспорить', 'решение'),
 ('рассматривать', 'дело'),
 ('доказать', 'немцов'),
 ('использовать', 'возможность'),
 ('обжаловать', 'решение'),
 ('получить', 'деньга'),
 ('пройти', 'этап'),
 ('рассмотреть', 

In [113]:
len(accs)

445

## 2. Метрики log-likelihood, dice, PMI

In [114]:
import nltk
import nltk.collocations

In [115]:
measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(accs)

In [116]:
#PMI
pmi = finder.nbest(measures.pmi, 100)
pmi[:10]

[('делать', 'вид'),
 ('сообщить', 'сведение'),
 ('стать', 'обыск'),
 ('арестовывать', 'счёт'),
 ('объявить', 'перерыв'),
 ('назначить', 'наказание'),
 ('обвинить', 'военнослужащий'),
 ('обвинить', 'лидер'),
 ('обвинить', 'президент'),
 ('обвинить', 'сотрудник')]

In [117]:
#log-likelihood
logl = finder.nbest(measures.likelihood_ratio, 100)
logl[:10]

[('обжаловать', 'решение'),
 ('передать', 'дело'),
 ('запретить', 'деятельность'),
 ('выплатить', 'компенсация'),
 ('назначить', 'наказание'),
 ('рассмотреть', 'вопрос'),
 ('подать', 'иск'),
 ('предъявить', 'обвинение'),
 ('рассматривать', 'дело'),
 ('вынести', 'приговор')]

In [118]:
#dice
dice = finder.nbest(measures.dice, 100)
dice[:10]

[('делать', 'вид'),
 ('объявить', 'перерыв'),
 ('сообщить', 'сведение'),
 ('стать', 'обыск'),
 ('назначить', 'наказание'),
 ('запретить', 'деятельность'),
 ('арестовывать', 'счёт'),
 ('передать', 'дело'),
 ('решить', 'проблема'),
 ('предъявить', 'обвинение')]

## 3. Золотой стандарт

In [119]:
standard = set(pmi).intersection(set(dice)).intersection(set(dice))

In [120]:
len(list(standard))

72

In [121]:
with open("verb_coll.txt", "r", encoding='utf-8') as d:
    lines = d.readlines()

In [122]:
dict_coll = []

for line in lines:
    if 'VN' in line:
        new_line = line.split('VN\t')[1]
        arr = [m.parse(t)[0].normal_form for t in new_line.split()]
        
        dict_coll.append(tuple(arr))

In [123]:
dict_coll[:10]

[('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария'),
 ('завоевать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('пользоваться', 'авторитет'),
 ('пользоваться', 'больший', 'авторитет')]

In [126]:
#Пересечение со словарем глагольной сочетаемости
golden_standard = standard.intersection(set(dict_coll))
golden_standard

{('делать', 'вид'),
 ('доказать', 'вина'),
 ('доказать', 'правота'),
 ('запретить', 'использование'),
 ('назначить', 'срок'),
 ('направить', 'запрос'),
 ('объявить', 'перерыв'),
 ('передать', 'дело'),
 ('предъявить', 'обвинение'),
 ('пройти', 'курс'),
 ('пройти', 'проверка'),
 ('решить', 'проблема'),
 ('решить', 'судьба')}

## 4. Добавление коллокаций

In [127]:
standard.difference(golden_standard)

{('арестовывать', 'акция'),
 ('арестовывать', 'счёт'),
 ('взыскать', 'задолженность'),
 ('взыскать', 'неустойка'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('запретить', 'зао'),
 ('иметь', 'организация'),
 ('иметь', 'повод'),
 ('иметь', 'последствие'),
 ('иметь', 'резонанс'),
 ('иметь', 'слово'),
 ('использовать', 'женщинсмертница'),
 ('использовать', 'знак'),
 ('использовать', 'льгота'),
 ('использовать', 'отсрочка'),
 ('использовать', 'поправка'),
 ('назначить', 'наказание'),
 ('направить', 'альтернативщик'),
 ('направить', 'документ'),
 ('направить', 'материал'),
 ('направить', 'представитель'),
 ('обвинить', 'военнослужащий'),
 ('обвинить', 'лидер'),
 ('обвинить', 'президент'),
 ('обвинить', 'сотрудник'),
 ('обязать', 'ведомство'),
 ('обязать', 'депутат'),
 ('обязать', 'комиссия'),
 ('обязать', 'ответчик'),
 ('обязать', 'руководитель'),
 ('оспаривать', 'отказ'),
 ('оставить', 'автограф'),
 ('оставить', 'заявление'),
 ('оставить'

Как мне кажется, словосочетания ниже являются коллокациями (т.к. едва ли можно представить сочетания данных существительных с другими глаголами так, чтобы выражался тот же смысл) и их стоить добавить в золотой стандарт. Они отсутствуют в словаре в силу своей специфичности и узконаправленности в смысле сферы применения (судебная лексика). 

In [128]:
to_add = {('взыскать', 'задолженность'), ('взыскать', 'неустойка'), 
          ('оспаривать', 'отказ'), ('пройти', 'прение'), ('просить', 'убежище')}

In [129]:
golden_standard = golden_standard.union(to_add)

## 5. Ранговая корреляция